In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [3]:
#Gaussian Naive Bayes

salary = spark.read.option("inferschema","true").option("header","true").csv("nv.csv")
salary.show(5)

+---+------+---------+
|Age|Salary|Purchased|
+---+------+---------+
| 59| 88000|        1|
| 35| 61000|        0|
| 37| 70000|        1|
| 52| 21000|        1|
| 48|141000|        0|
+---+------+---------+
only showing top 5 rows



In [4]:
salary.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Purchased: integer (nullable = true)



In [5]:
means=salary.groupBy("Purchased").mean("Salary")
means.show()

+---------+-----------------+
|Purchased|      avg(Salary)|
+---------+-----------------+
|        1|          82300.0|
|        0|77142.85714285714|
+---------+-----------------+



In [6]:
means=salary.groupBy("Purchased").mean("Age")
means.show()

+---------+-----------------+
|Purchased|         avg(Age)|
+---------+-----------------+
|        1|             45.5|
|        0|38.42857142857143|
+---------+-----------------+



In [8]:
variances=salary.groupBy("Purchased").agg(variance("Salary"))
variances.show()

+---------+--------------------+
|Purchased|    var_samp(Salary)|
+---------+--------------------+
|        1|1.3173444444444444E9|
|        0| 8.628095238095237E8|
+---------+--------------------+



In [9]:
variances=salary.groupBy("Purchased").agg(variance("Age"))
variances.show()

+---------+------------------+
|Purchased|     var_samp(Age)|
+---------+------------------+
|        1| 67.16666666666667|
|        0|21.952380952380953|
+---------+------------------+



In [11]:
salary_1=salary.where(col("Purchased")==1)
salary_0=salary.where(col("Purchased")==0)

salary_1.show()
salary_0.show()

+---+------+---------+
|Age|Salary|Purchased|
+---+------+---------+
| 59| 88000|        1|
| 37| 70000|        1|
| 52| 21000|        1|
| 37| 93000|        1|
| 48|138000|        1|
| 37| 78000|        1|
| 39|134000|        1|
| 49| 89000|        1|
| 55| 39000|        1|
| 42| 73000|        1|
+---+------+---------+

+---+------+---------+
|Age|Salary|Purchased|
+---+------+---------+
| 35| 61000|        0|
| 48|141000|        0|
| 37| 62000|        0|
| 41| 79000|        0|
| 37| 77000|        0|
| 35| 57000|        0|
| 36| 63000|        0|
+---+------+---------+



In [12]:
n=salary.count()
print(n)

17


In [13]:
n_1=salary_1.count()
n_0=salary_0.count()
print(n_1,n_0)


10 7


In [14]:
p_1=n_1/n
p_0=n_0/n
print(p_1,p_0)

0.5882352941176471 0.4117647058823529


# using collect to convert expanded dataframe to single 

In [15]:
salary.groupBy("Purchased").agg(variance("Salary")).collect()

[Row(Purchased=1, var_samp(Salary)=1317344444.4444444),
 Row(Purchased=0, var_samp(Salary)=862809523.8095237)]

In [16]:
mean_0=salary.groupBy("Purchased").mean("Salary").collect()[0][1]
mean_1=salary.groupBy("Purchased").mean("Salary").collect()[1][1]
print(mean_0)
print(mean_1)



82300.0
77142.85714285714


In [22]:
mean_0a=salary.groupBy("Purchased").mean("Age").collect()[0][1]
mean_1a=salary.groupBy("Purchased").mean("Age").collect()[1][1]
print(mean_0a)
print(mean_1a)


45.5
38.42857142857143


In [27]:
var_0=salary.groupBy("Purchased").agg(stddev("Salary")).collect()[0][1]
var_1=salary.groupBy("Purchased").agg(stddev("Salary")).collect()[1][1]
print(var_0)
print(var_1)


36295.239969511764
29373.61952176687


In [28]:
var_0a=salary.groupBy("Purchased").agg(stddev("Age")).collect()[0][1]
var_1a=salary.groupBy("Purchased").agg(stddev("Age")).collect()[1][1]
print(var_0a)
print(var_1a)

8.195527235429498
4.68533680244878


In [ ]:
#p(1|38,71000) = p(1) * p(38|1) * p(71000|1)

In [34]:
import math
p_38_1=((1/math.sqrt(2*math.pi*var_1a))*math.exp(-0.5*((38-mean_1a)**2)/var_1a))
p_38_1

0.18072869105321568

In [35]:
p_71000_1=((1/math.sqrt(2*math.pi*var_1a))*math.exp(-0.5*((71000-mean_1a)**2)/var_1a))
p_71000_1

0.0

In [36]:
p_1_38_71000 = p_1 * p_38_1 * p_71000_1
p_1_38_71000

0.0

In [37]:
((1/math.sqrt(2*math.pi*8.2))*math.exp(-0.5*((38-45.5)**2)/8.2))

0.0045125915875449014

In [40]:
((1/math.sqrt(2*math.pi*36295.24))*math.exp(-0.5*((71000-82300)**2)/36295.24))

0.0